In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext

In [2]:
spark = SparkSession.builder.appName('task9').getOrCreate()

In [18]:
# Load the dataset
sc = SparkContext.getOrCreate()
employee_rdd = sc.textFile('employees.csv')
employee_rdd = employee_rdd.map(lambda x: x.split(","))
employee_rdd.take(5)

[['EMPLOYEE_ID',
  'FIRST_NAME',
  'LAST_NAME',
  'EMAIL',
  'PHONE_NUMBER',
  'HIRE_DATE',
  'JOB_ID',
  'SALARY',
  'COMMISSION_PCT',
  'MANAGER_ID',
  'DEPARTMENT_ID'],
 ['198',
  'Donald',
  'OConnell',
  'DOCONNEL',
  '650.507.9833',
  '21-JUN-07',
  'SH_CLERK',
  '2600',
  ' - ',
  '124',
  '50'],
 ['199',
  'Douglas',
  'Grant',
  'DGRANT',
  '650.507.9844',
  '13-JAN-08',
  'SH_CLERK',
  '2600',
  ' - ',
  '124',
  '50'],
 ['200',
  'Jennifer',
  'Whalen',
  'JWHALEN',
  '515.123.4444',
  '17-SEP-03',
  'AD_ASST',
  '4400',
  ' - ',
  '101',
  '10'],
 ['201',
  'Michael',
  'Hartstein',
  'MHARTSTE',
  '515.123.5555',
  '17-FEB-04',
  'MK_MAN',
  '13000',
  ' - ',
  '100',
  '20']]

In [19]:
# Count the number of records
employee_rdd.count()-1

50

In [21]:
# filter the employess on joined date
from datetime import datetime
header = employee_rdd.first()
data_rdd = employee_rdd.filter(lambda x: x != header)
filtered_rdd = data_rdd.filter(lambda x: datetime.strptime(x[5], "%d-%b-%y").year > 2005)
filtered_rdd.count()

20

In [23]:
# Map employee name
names_rdd = data_rdd.map(lambda x: x[1]+" "+x[2])
names_rdd.take(5)

['Donald OConnell',
 'Douglas Grant',
 'Jennifer Whalen',
 'Michael Hartstein',
 'Pat Fay']

In [28]:
# aggregate by department
dept_rdd = data_rdd.map(lambda x: (x[10],int(x[7])))
aggr_rdd = dept_rdd.mapValues(lambda x: (x,1)).reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1]))
aggr_rdd = aggr_rdd.mapValues(lambda x: x[0]/x[1])
aggr_rdd.collect()

[('10', 4400.0),
 ('40', 6500.0),
 ('70', 10000.0),
 ('110', 10154.0),
 ('90', 19333.333333333332),
 ('50', 3721.7391304347825),
 ('20', 9500.0),
 ('60', 5760.0),
 ('100', 8601.333333333334),
 ('30', 4150.0)]